In [1]:
import csv
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_json('reviews_Movies_and_TV_5.json', lines=True)
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0005019281,"[0, 0]",4,This is a charming version of the classic Dick...,"02 26, 2008",ADZPIG9QOCDG5,"Alice L. Larson ""alice-loves-books""",good version of a classic,1203984000
1,0005019281,"[0, 0]",3,It was good but not as emotionally moving as t...,"12 30, 2013",A35947ZP82G7JH,Amarah Strack,Good but not as moving,1388361600
2,0005019281,"[0, 0]",3,"Don't get me wrong, Winkler is a wonderful cha...","12 30, 2013",A3UORV8A9D5L2E,Amazon Customer,Winkler's Performance was ok at best!,1388361600
3,0005019281,"[0, 0]",5,Henry Winkler is very good in this twist on th...,"02 13, 2008",A1VKW06X1O2X7V,"Amazon Customer ""Softmill""",It's an enjoyable twist on the classic story,1202860800
4,0005019281,"[0, 0]",4,This is one of the best Scrooge movies out. H...,"12 22, 2013",A3R27T4HADWFFJ,BABE,Best Scrooge yet,1387670400


In [25]:
df2 = df[pd.notnull(df['reviewText'])]

1088752

In [26]:
df_asin_review_text = df2[['asin','reviewText']]
df_asin_review_count = df_asin_review_text.groupby("asin", as_index=False).count()

df_knn = df_asin_review_count[df_asin_review_count.reviewText >=100]
df_sentiment = df_asin_review_count[df_asin_review_count.reviewText  < 100]

final_df_knn = df2.loc[df2['asin'].isin(df_knn['asin'])]

final_df_sentiment = df2.loc[df2['asin'].isin(df_sentiment['asin'])]

final_df_knn.to_json('final_df_knn.json')
final_df_sentiment.to_json('final_df_sentiment.json')

In [27]:
len(final_df_knn)

809924

In [28]:
len(final_df_sentiment)

887609

In [29]:
len(df2)

final_df_sentiment.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
109,0005119367,"[1, 1]",4,"A truly wonderfully made film, mostly accurate...","07 6, 2006",AJSYTZQVJJOYW,A. Abela,Wonderful!!!,1152144000
110,0005119367,"[1, 1]",5,Joseph is amazing story to read and watch. A ...,"05 1, 2010",A1QNGKV688Q8TE,Aaron D. Koehler,Joseph,1272672000
111,0005119367,"[4, 4]",5,This WONDERFUL film is a total treat on every ...,"03 15, 2005",A36311XF4O81IK,A. J. Trivette,As real as it gets!,1110844800
112,0005119367,"[102, 106]",5,Though this interpretation of Joseph's life ha...,"02 21, 2004",AQ8DU6XVA3USJ,"Alejandra Vernon ""artist & illustrator""",a man of character and faith,1077321600
113,0005119367,"[1, 1]",5,I loved this story about Joseph. It's one of m...,"01 22, 2009",A1ZLFCUF4XX7RZ,Alex,True Faith,1232582400


In [30]:

final_df_sentiment = final_df_sentiment.reindex(columns=['asin', 'helpful', 'overall', 'reviewText', 'reviewTime', 'reviewerID', 'reviewerName','summary','unixReviewTime','numerator','denominator'])
final_df_sentiment.head()


# helpful_df = pd.DataFrame(final_df_sentiment.helpful.values.tolist(), columns=['numerator', 'denominator'],index=final_df_sentiment.index)
# helpful_df.head()
# final_df_sentiment[['helpful','total']] = pd.DataFrame(final_df_sentiment.helpful.values.tolist(), index=final_df_sentiment.index)

#final_df_sentiment = final_df_sentiment[final_df_sentiment['total'] > 5])

# helpfulness = final_df_sentiment[['helpful','asin']]
# df3 = pd.DataFrame(helpfulness)
# df3.head()
# df3[['helpful','total']] = pd.DataFrame(df3.helpful.values.tolist(), index=df3.index)
# dfhelpful = df3[['asin', 'helpful','total']]
# temp_dfhelp = dfhelpful[dfhelpful['total']!=0]
# temp_dfhelp = (temp_dfhelp[temp_dfhelp['total'] > 5])
# temp_dfhelp = (temp_dfhelp[(temp_dfhelp['helpful'] / temp_dfhelp['total']) > 0.8 ])

# final_df_sentiment_1 = final_df_sentiment.loc[final_df_sentiment['asin'].isin(temp_dfhelp['asin'])]
# len(final_df_sentiment_1)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,numerator,denominator
109,0005119367,"[1, 1]",4,"A truly wonderfully made film, mostly accurate...","07 6, 2006",AJSYTZQVJJOYW,A. Abela,Wonderful!!!,1152144000,NaN,NaN
110,0005119367,"[1, 1]",5,Joseph is amazing story to read and watch. A ...,"05 1, 2010",A1QNGKV688Q8TE,Aaron D. Koehler,Joseph,1272672000,NaN,NaN
111,0005119367,"[4, 4]",5,This WONDERFUL film is a total treat on every ...,"03 15, 2005",A36311XF4O81IK,A. J. Trivette,As real as it gets!,1110844800,NaN,NaN
112,0005119367,"[102, 106]",5,Though this interpretation of Joseph's life ha...,"02 21, 2004",AQ8DU6XVA3USJ,"Alejandra Vernon ""artist & illustrator""",a man of character and faith,1077321600,NaN,NaN
113,0005119367,"[1, 1]",5,I loved this story about Joseph. It's one of m...,"01 22, 2009",A1ZLFCUF4XX7RZ,Alex,True Faith,1232582400,NaN,NaN


In [31]:
final_df_sentiment[['numerator','denominator']] = pd.DataFrame(final_df_sentiment.helpful.values.tolist(), index= final_df_sentiment.index)

In [33]:
final_df_sentiment.to_json('final_df_sentiment.json')

In [40]:
final_df_sentiment = (final_df_sentiment[final_df_sentiment['denominator'] > 0])
final_df_sentiment = final_df_sentiment[final_df_sentiment['numerator'] / final_df_sentiment['denominator'] >= 0.8] 

In [42]:
final_df_sentiment.to_json('final_df_sentiment.json')

In [43]:
rating_review = final_df_sentiment[['reviewText','overall']]

In [46]:
rating_review.head()

,reviewText,overall
109,"A truly wonderfully made film, mostly accurate...",4
110,Joseph is amazing story to read and watch. A ...,5
111,This WONDERFUL film is a total treat on every ...,5
112,Though this interpretation of Joseph's life ha...,5
113,I loved this story about Joseph. It's one of m...,5


In [47]:
rating_review["opinion"] = rating_review["overall"].apply(lambda value: "good" if value >= 3 else "bad")
final_df_knn["opinion"] = final_df_knn["overall"].apply(lambda value: "good" if value >= 3 else "bad")

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
rating_review[rating_review.overall == 4].head(3)

,reviewText,overall,opinion
109,"A truly wonderfully made film, mostly accurate...",4,good
166,"I liked this movie, except one scene where Pot...",4,good
191,This four hour video looks a the familiar &quo...,4,good


In [51]:
rating_review["reviewText"].head(3)

109    A truly wonderfully made film, mostly accurate...
110    Joseph is amazing story to read and watch.  A ...
111    This WONDERFUL film is a total treat on every ...
Name: reviewText, dtype: object

In [55]:
regular_expression = re.compile('[^a-z]+')
def process_text(review_text):
    review_text = review_text.lower()
    review_text = regular_expression.sub(' ', review_text).strip()
    return review_text

rating_review["processedReview"] = rating_review["reviewText"].apply(process_text)
final_df_knn["processedReview"] = final_df_knn["reviewText"].apply(process_text)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [59]:
final_df_knn.to_json('final_df_knn_after_removing_spaces.json')
rating_review.to_json('rating_review.json')

In [61]:
rating_review = rating_review.drop(['reviewText', 'overall'],1)

In [62]:
rating_review.head()

,opinion,processedReview
109,good,a truly wonderfully made film mostly accurate ...
110,good,joseph is amazing story to read and watch a pe...
111,good,this wonderful film is a total treat on every ...
112,good,though this interpretation of joseph s life ha...
113,good,i loved this story about joseph it s one of my...


In [63]:
train_data = rating_review

In [64]:
(len(train_data), len(final_df_knn))

(304084, 809924)

In [73]:
print len(train_data)

304084


In [65]:
#CountVectorizer to get bag of words model
count_vectorizer = CountVectorizer(min_df = 1, ngram_range = (1, 3))
X_train_bags = count_vectorizer.fit_transform(train_data["processedReview"])

#tfidf to bag of words
tfidf_transformer = TfidfTransformer()
X_train_data = tfidf_transformer.fit_transform(X_train_bags)

Y_train_data = train_data["opinion"]
Y_test_knn_data = final_df_knn["opinion"]
predicted_output = dict();

In [66]:
X_test_counts = count_vectorizer.transform(final_df_knn["processedReview"])
X_test_knn_data = tfidf_transformer.transform(X_test_counts)

In [98]:
def save_sparse_csr(filename, array):
    # note that .npz extension is added automatically
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)

def load_sparse_csr(filename):
    # here we need to add .npz extension manually
    loader = np.load(filename + '.npz')
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])

save_sparse_csr("X_train_data",X_train_data)
save_sparse_csr("X_test_knn_data",X_test_knn_data)
Y_test_knn_data.to_csv("Y_test_knn_data.csv")

In [105]:
Y_train_data.to_csv("Y_train_data.csv")

In [102]:
# Y_train_data = Series.from_csv('Y_train_data.csv')
# X_train_data = load_sparse_csr("X_train_data")
# X_test_knn_data = load_sparse_csr("X_test_knn_data")
# Y_test_knn_data = Series.from_csv('Y_test_knn_data.csv')

In [106]:
type(Y_train_data)

pandas.core.series.Series

In [72]:
logistic_regres = LogisticRegression(C=1e5)
logistic_regres.fit(X_train_data, Y_train_data)
predicted_output['logistic'] = logistic_regres.predict(X_test_knn_data)

In [75]:
accuracy_score(Y_test_knn_data, predicted_output['logistic'])

0.9248275146803898

In [83]:
output = logistic_regres.predict_proba(X_test_knn_data)

In [86]:
output_in_5  = output[:,1]*5

In [87]:
# predicted_rating = test_output['logistic'][:,1]*5
final_df_knn["predicted_rating"] = output_in_5;
final_df_knn.head()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,opinion,processedReview,predicted_rating
0,0005019281,"[0, 0]",4,This is a charming version of the classic Dick...,"02 26, 2008",ADZPIG9QOCDG5,"Alice L. Larson ""alice-loves-books""",good version of a classic,1203984000,good,this is a charming version of the classic dick...,5.000000
1,0005019281,"[0, 0]",3,It was good but not as emotionally moving as t...,"12 30, 2013",A35947ZP82G7JH,Amarah Strack,Good but not as moving,1388361600,good,it was good but not as emotionally moving as t...,4.999790
2,0005019281,"[0, 0]",3,"Don't get me wrong, Winkler is a wonderful cha...","12 30, 2013",A3UORV8A9D5L2E,Amazon Customer,Winkler's Performance was ok at best!,1388361600,good,don t get me wrong winkler is a wonderful char...,4.617876
3,0005019281,"[0, 0]",5,Henry Winkler is very good in this twist on th...,"02 13, 2008",A1VKW06X1O2X7V,"Amazon Customer ""Softmill""",It's an enjoyable twist on the classic story,1202860800,good,henry winkler is very good in this twist on th...,4.999957
4,0005019281,"[0, 0]",4,This is one of the best Scrooge movies out. H...,"12 22, 2013",A3R27T4HADWFFJ,BABE,Best Scrooge yet,1387670400,good,this is one of the best scrooge movies out hen...,5.000000


In [88]:
final_df_knn.to_json('final_result_after_logistic.json')

In [112]:
final_df_knn.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,opinion,processedReview,predicted_rating
0,0005019281,"[0, 0]",4,This is a charming version of the classic Dick...,"02 26, 2008",ADZPIG9QOCDG5,"Alice L. Larson ""alice-loves-books""",good version of a classic,1203984000,good,this is a charming version of the classic dick...,5.000000
1,0005019281,"[0, 0]",3,It was good but not as emotionally moving as t...,"12 30, 2013",A35947ZP82G7JH,Amarah Strack,Good but not as moving,1388361600,good,it was good but not as emotionally moving as t...,4.999790
2,0005019281,"[0, 0]",3,"Don't get me wrong, Winkler is a wonderful cha...","12 30, 2013",A3UORV8A9D5L2E,Amazon Customer,Winkler's Performance was ok at best!,1388361600,good,don t get me wrong winkler is a wonderful char...,4.617876
3,0005019281,"[0, 0]",5,Henry Winkler is very good in this twist on th...,"02 13, 2008",A1VKW06X1O2X7V,"Amazon Customer ""Softmill""",It's an enjoyable twist on the classic story,1202860800,good,henry winkler is very good in this twist on th...,4.999957
4,0005019281,"[0, 0]",4,This is one of the best Scrooge movies out. H...,"12 22, 2013",A3R27T4HADWFFJ,BABE,Best Scrooge yet,1387670400,good,this is one of the best scrooge movies out hen...,5.000000
